In [4]:
import re
import os
import pickle
import keras
import statistics

import numpy as np
import pandas as pd

from os import listdir
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import sequence

COUNTRY_CODES = ['CN','CZ','FR','AT','JP','TR','DK','DE','HU','US','IE','GB','BA','PL','PT','IT','MT','IN','ES','PH',
'MG','TH','PK','AU','TW','SE','LT','VN','NL','CA','CD','MX','BE','DZ','BR','CH','ID','CL','KR','RO','LA','TN','FI','AF',
'AX','AL','AS','AD','AO','AI','AQ','AG','AR','AM','AW','AZ','BS','BH','BD','BB','BY','BZ','BJ','BM','BT','BO','BQ','BW',
'BV','IO','BN','BG','BF','BI','KH','CM','CV','KY','CF','TD','CX','CC','CO','KM','CG','CK','CR','CI','HR','CU','CW','CY',
'DJ','DM','DO','EC','EG','SV','GQ','ER','EE','ET','FK','FO','FJ','GF','PF','TF','GA','GM','GE','GH','GI','GR','GL','GD',
'GP','GU','GT','GG','GN','GW','GY','HT','HM','VA','HN','HK','IS','IR','IQ','IM','IL','JM','JE','JO','KZ','KE','KI','KP',
'KW','KG','LV','LB','LS','LR','LY','LI','LU','MO','MK','MW','MY','MV','ML','MH','MQ','MR','MU','YT','FM','MD','MC','MN',
'ME','MS','MA','MZ','MM','NA','NR','NP','NC','NZ','NI','NE','NG','NU','NF','MP','NO','OM','PW','PS','PA','PG','PY','PE',
'PN','PR','QA','RE','RU','RW','BL','SH','KN','LC','MF','PM','VC','WS','SM','ST','SA','SN','RS','SC','SL','SG','SX','SK',
'SI','SB','SO','ZA','GS','SS','LK','SD','SR','SJ','SZ','SY','TJ','TZ','TL','TG','TK','TO','TT','TM','TC','TV','UG','UA',
'AE','UM','UY','UZ','VU','VE','VG','VI','WF','EH','YE','ZM','ZW']

EAN_REGEX = '(?<=\s)\d{13}(?=\s)'


MODEL_1_DIR = r'C:\Users\mail\Downloads\data\Parcellet\models_revamp\model 1\OTHER_NOTOTHER_OVER_WORDEMB.h5'
TOKENIZER_1_DIR = r'C:\Users\mail\Downloads\data\Parcellet\models_revamp\model 1\OTHER_NOTOTHER_OVER_WORDEMB.pkl'
MODEL_2_DIR = r'C:\Users\mail\Downloads\data\Parcellet\models_revamp\model 2\NAMECOLOR_PRICESIZE_OVER_WORDEMB.h5'
TOKENIZER_2_DIR = r'C:\Users\mail\Downloads\data\Parcellet\models_revamp\model 2\NAMECOLOR_PRICESIZE_OVER_WORDEMB.pkl'
MODEL_2B_DIR = r'C:\Users\mail\Downloads\data\Parcellet\models_revamp\model 2b\NAME_COLOR_PRICE_SIZE_OVER_WORDEMB.h5'
TOKENIZER_2B_DIR = r'C:\Users\mail\Downloads\data\Parcellet\models_revamp\model 2b\NAME_COLOR_PRICE_SIZE_OVER_WORDEMB.pkl'
MODEL_3_DIR = r'C:\Users\mail\Downloads\data\Parcellet\models_revamp\model 3\NAME_COLOR_OVER_WORDEMB.h5'
TOKENIZER_3_DIR = r'C:\Users\mail\Downloads\data\Parcellet\models_revamp\model 3\NAME_COLOR_OVER_WORDEMB.pkl'
MODEL_4_DIR = r'C:\Users\mail\Downloads\data\Parcellet\models_revamp\model 4\PRICE_SIZE_OVER_WORDEMB.h5'
TOKENIZER_4_DIR = r'C:\Users\mail\Downloads\data\Parcellet\models_revamp\model 4\PRICE_SIZE_OVER_WORDEMB.pkl'

THRESHOLD = 0.99
SAMPLE_AMOUNT = 100
RANDOM_STATE = 7
TARGETS_1 = ['OTHER', 'NOT OTHER']
model_1 = keras.models.load_model(MODEL_1_DIR)
tokenizer_1 = pickle.load(open(TOKENIZER_1_DIR, 'rb'))
TARGETS_2 = ['NAME COLOR', 'PRICE SIZE']
model_2 = keras.models.load_model(MODEL_2_DIR)
tokenizer_2 = pickle.load(open(TOKENIZER_2_DIR, 'rb'))
TARGETS_2B = ['NAME', 'COLOR', 'PRICE', 'SIZE']
model_2B = keras.models.load_model(MODEL_2B_DIR)
tokenizer_2B = pickle.load(open(TOKENIZER_2B_DIR, 'rb'))
TARGETS_3 = ['NAME', 'COLOR']
model_3 = keras.models.load_model(MODEL_3_DIR)
tokenizer_3 = pickle.load(open(TOKENIZER_3_DIR, 'rb'))
TARGETS_4 = ['PRICE', 'SIZE']
model_4 = keras.models.load_model(MODEL_3_DIR)
tokenizer_4 = pickle.load(open(TOKENIZER_3_DIR, 'rb'))
le = LabelEncoder()
print('Model, Tokenizer and LabelEncoder loaded.')

def predictClass(text, tok, model, targets):
    text_pad = sequence.pad_sequences(tok.texts_to_sequences([text]), maxlen=300)
    predict_x = model.predict(text_pad)
    predict_class = np.argmax(predict_x, axis=1)
    le.fit_transform(targets)
    score = le.inverse_transform(predict_class)
    prediction = score[0]
    return prediction

PATH = r'C:\Users\mail\Downloads\data\Parcellet\Eval'
list_files = listdir(PATH)
k = 0

percent_list = []
column_amounts = []
correct_amounts = []

def evaluate(num):
    correct = 0
    dataset_filename = os.listdir(PATH)[num]
    print('________________________________________________________________________')
    print('----------{}----------'.format(dataset_filename))
    dataset_path = os.path.join("../..", PATH, dataset_filename)
    df = pd.read_csv(dataset_path, error_bad_lines=False, engine='c',
                 encoding='ISO-8859-14', low_memory=False, dtype=str)
    if len(df.index) >= SAMPLE_AMOUNT:
        try:
            df = df.sample(SAMPLE_AMOUNT,
                           random_state=RANDOM_STATE)
        except ValueError:
            pass
    df = df.reset_index(drop=True)
    column_headers = list(df.columns)
    # list of list of predictions including original column name
    predictions_map = list(map(lambda item: [item], column_headers))

    # list of list of predictions only - populate now then popped later
    predictions_only = list(map(lambda item: [item], column_headers))

    map_list = []
    maps_object = []
    Predictions = []
    Certainty = []
    manual_maps_list = []

    for n in range(len(column_headers)):
        print('----------Mapping column {num} of {len}----------'
              .format(num=n+1, len=len(column_headers)))
        df_select = df[column_headers[n]]
        # sequence padding and tokenization of data requires string type
        df_select = df_select.astype(str)
        multiple_predictions = []
        multiple_certainties = []

        for m in range(len(df_select)):
            class_predictions =[]
            data = df_select[m]
            model_1_prediction = predictClass(data, tokenizer_1, model_1, TARGETS_1)
            print('Model 1 Pred:', model_1_prediction)
            model_2_prediction = predictClass(data, tokenizer_2, model_2, TARGETS_2)
            print('Model 2 Pred:', model_2_prediction)
            model_2b_prediction = predictClass(data, tokenizer_2B, model_2B, TARGETS_2B)
            print('Model 2b Pred:', model_2b_prediction)
            model_3_prediction = predictClass(data, tokenizer_3, model_3, TARGETS_3)
            #print('Model 3 Pred:', model_3_prediction)
            model_4_prediction = predictClass(data, tokenizer_4, model_4, TARGETS_4)
            #print('Model 4 Pred:', model_4_prediction)
            if model_1_prediction == 'OTHER':
               predicted_class = 'OTHER'
               predictions_map[n].append(predicted_class)
               predictions_only[n].append(predicted_class)
            elif model_1_prediction == 'NOT OTHER':
                if model_2_prediction == 'NAME COLOR' and model_2b_prediction == model_3_prediction:
                    predicted_class = model_3_prediction
                    predictions_map[n].append(predicted_class)
                    predictions_only[n].append(predicted_class)
                if model_2_prediction == 'PRICE SIZE' and model_2b_prediction == model_3_prediction:
                    predicted_class = model_4_prediction
                    predictions_map[n].append(predicted_class)
                    predictions_only[n].append(predicted_class)
                if model_2_prediction == 'PRICE SIZE' and (model_2b_prediction != 'PRICE' or model_2b_prediction != 'SIZE'):
                    predicted_class = model_3_prediction
                    predictions_map[n].append(predicted_class)
                    predictions_only[n].append(predicted_class)
                else:
                    print('Predictions only Other left.')
                    predicted_class = 'OTHER'
                    predictions_map[n].append(predicted_class)
                    predictions_only[n].append(predicted_class)

            print('Data: {data}, Class Prediction: {prediction}'
                  .format(data=data, prediction=predicted_class))

        predictions_only[n].pop(0)
        column_predictions = (predictions_only[n])
        column_predictions_series = pd.Series(column_predictions)
        column_predictions_count = column_predictions_series.value_counts()
        print('----------Column Mapping Summary----------')
        print('column predictions:', column_predictions)
        #print('column predictions series:', column_predictions_series)
        #print('column predictions count:', column_predictions_count)
        print('predicted class:', column_predictions_count.index[0])
        print('number of classifications:', column_predictions_count.iloc[0])
        #print('actual class:', manual_maps[num][n])

        if column_predictions_count.iloc[0] > len(df_select) * THRESHOLD:
            print(column_predictions_count.index[0], 'is the Majority Predicted Class.')
            print('The majority class is: {pred}; {num_pred} of {len} predictions.'
                  '\nOriginal Class: {origin}'
                  .format(pred=column_predictions_count.index[0],
                          num_pred=column_predictions_count.iloc[0],
                          len=len(df_select),
                          origin=column_headers[n]))
            Predictions.append(column_predictions_count.index[0])
            Certainty.append(100)

        else:
            print('There is no Majority Predicted Class above the threshold.'
                  '\nOriginal Class: {origin}'
                  .format(origin=column_headers[n]))
            for i in range(len(column_predictions_count)):
                print('Predicted class {i}: {pred}; {num_pred} of {len} predictions.'
                      .format(i=i+1,
                              pred=column_predictions_count.index[i],
                              num_pred=column_predictions_count.iloc[i],
                              len=len(df_select)))
                multiple_predictions.append(column_predictions_count.index[i])
                multiple_certainties.append(column_predictions_count.iloc[i] / len(df_select) * 100)
            Predictions.append(multiple_predictions)
            Certainty.append(multiple_certainties)
        #if column_predictions_count.index[0] == manual_maps[num][n]:
        #        correct +=1
        print('________________________________________________')

    print('----------Dataset Mapping Summary----------')
    #percent = correct/len(column_headers)*100
    #print('Correctly mapped columns: {correct}/{total}'.format(correct=correct, total=len(column_headers)))
    #print('Correctly mapped column percentage: {percent}'.format(percent=percent))
    #percent_list.append(percent)
    print('map list:', predictions_map)
    print('predictions only:', predictions_only)
    #for item in range(len(manual_maps[num])):
    #    temp = manual_maps[num][item]
    #    manual_maps_list.append(temp)
    #print('manual map:', manual_maps_list)
    print('Predictions:', Predictions)
    #print('Certainties:', Certainty)
    #column_amounts.append(len(column_headers))
    #correct_amounts.append(correct)





Model, Tokenizer and LabelEncoder loaded.


In [5]:
manual_map_A_dataset = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER', 4: 'PROD_BARCODE_NUM', 5: 'OTHER',
                        6: 'OTHER', 7: 'OTHER', 8: 'OTHER', 9: 'OTHER', 10: 'OTHER', 11: 'OTHER', 12 : 'OTHER',
                        13: 'OTHER', 14: 'OTHER', 15: 'OTHER', 16: 'OTHER', 17: 'OTHER'}
manual_map_A_Discount_bigbuy_da = {0: 'PROD_NUM', 1: 'OTHER', 2: 'PROD_NAME', 3: 'OTHER', 4: 'OTHER', 5: 'OTHER'}
manual_map_A_Discount_bigbuy_en = {0: 'PROD_NUM', 1: 'OTHER', 2: 'PROD_NAME', 3: 'OTHER', 4: 'OTHER', 5: 'OTHER'}
manual_map_A_Discount_Compressed1 = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'PROD_BARCODE_NUM', 4: 'OTHER',
                                     5: 'OTHER'}
manual_map_A_Discount_Compressed2 = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'PROD_BARCODE_NUM', 4:'OTHER'}
manual_map_A_Discount_presta_product_2399_da = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER',
                                                4: 'PROD_BARCODE_NUM', 5: 'OTHER', 6: 'OTHER'}
manual_map_A_Discount_presta_product_2403_en = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER', 4: 'OTHER',
                                                5: 'OTHER'}
manual_map_A_Discount_presta_product_2507_da = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER', 4: 'OTHER', 5:
    'PROD_BARCODE_NUM', 6: 'OTHER', 7: 'OTHER'}
manual_map_A_Discount_presta_product_2507_en = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER', 4: 'OTHER',
                                                5: 'PROD_BARCODE_NUM', 6: 'OTHER', 7: 'OTHER'}
manual_map_A_Discount_presta_product_2570_da = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER', 4: 'OTHER',
                                                5: 'PROD_BARCODE_NUM'}
manual_map_A_Discount_presta_product_2570_en = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER', 4: 'OTHER',
                                                5: 'PROD_BARCODE_NUM'}
manual_map_A_Discount_presta_product_2662_da = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER',
                                                4: 'PROD_BARCODE_NUM', 5: 'OTHER'}
manual_map_A_Discount_presta_product_2662_en = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER',
                                                4: 'PROD_BARCODE_NUM', 5: 'OTHER', 6: 'OTHER'}
manual_map_A_Discount_presta_product_2678_da = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER',
                                                4: 'PROD_BARCODE_NUM', 5: 'OTHER'}
manual_map_A_Discount_presta_product_2678_en = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER',
                                                4: 'PROD_BARCODE_NUM', 5: 'OTHER', 6: 'OTHER'}
manual_map_A_Discount_presta_product_3046_da = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER', 4: 'OTHER',
                                                5: 'PROD_NUM', 6: 'PROD_BARCODE_NUM', 7: 'OTHER', 8: 'OTHER'}
manual_map_A_Discount_presta_product_3046_en = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER', 4: 'OTHER',
                                                5: 'PROD_NUM', 6: 'PROD_BARCODE_NUM', 7: 'OTHER', 8: 'OTHER'}
manual_map_Bisgaard = {0: 'PROD_BARCODE_NUM', 1: 'OTHER', 2: 'OTHER', 3: 'OTHER', 4: 'OTHER', 5: 'OTHER', 6: 'OTHER', 7: 'OTHER',
             8: 'OTHER', 9: 'OTHER', 10: 'OTHER', 11: 'OTHER', 12: 'OTHER', 13: 'OTHER', 14: 'OTHER', 15: 'OTHER',
            16: 'OTHER', 17: 'OTHER', 18: 'OTHER', 19: 'OTHER', 20: 'OTHER', 21: 'OTHER', 22: 'OTHER', 23: 'OTHER',
            24: 'OTHER'}
manual_map_Scand = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'PROD_BARCODE_NUM', 4: 'OTHER', 5: 'OTHER', 6: 'OTHER',
                    7: 'OTHER', 8: 'OTHER', 9: 'OTHER', 10: 'OTHER', 11: 'OTHER', 12: 'OTHER'}
manual_map_items = {0: 'PROD_NUM', 1: 'PROD_BARCODE_NUM', 2:'PROD_NAME', 3: 'OTHER', 4: 'OTHER', 5: 'OTHER', 6: 'OTHER',
                    7: 'OTHER', 8: 'OTHER', 9: 'OTHER', 10: 'OTHER'}
manual_map_Joha = {0: 'OTHER', 1: 'OTHER', 2: 'OTHER', 3: 'OTHER', 4: 'OTHER', 5: 'PROD_BARCODE_NUM', 6: 'OTHER',
                   7: 'OTHER', 8: 'OTHER', 9: 'OTHER', 10: 'OTHER', 11: 'OTHER', 12: 'OTHER', 13: 'OTHER', 14: 'OTHER',
                   15: 'OTHER', 16: 'OTHER' }
manual_map_Modern_classic_upstart = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'PROD_BARCODE_NUM', 3: 'OTHER', 4: 'OTHER',
                                     5: 'OTHER', 6: 'OTHER', 7: 'OTHER'}
manual_map_PIF = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'PROD_BARCODE_NUM', 3: 'OTHER', 4: 'OTHER', 5: 'OTHER', 6: 'OTHER',
                  7: 'OTHER'}
manual_map_prisliste ={0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER', 4: 'OTHER', 5: 'OTHER', 6: 'OTHER',
                       7: 'OTHER', 8: 'OTHER', 9: 'OTHER', 10: 'OTHER', 11: 'OTHER', 12: 'OTHER', 13: 'OTHER',
                       14:'OTHER', 15: 'OTHER', 16: 'OTHER', 17: 'PROD_BARCODE_NUM', 18: 'OTHER'}
manual_map_VAREFIL = {0: 'PROD_NUM', 1: 'PROD_BARCODE_NUM', 2: 'PROD_NAME', 3: 'PROD_NAME', 4: 'OTHER',  5: 'OTHER',
                      6: 'OTHER'}
manual_map_Parcellet = {0: 'PROD_NUM', 1: 'PROD_NAME', 2: 'OTHER', 3: 'OTHER', 4: 'OTHER', 5: 'OTHER', 6: 'OTHER',
                        7: 'OTHER', 8: 'PROD_BARCODE_NUM', 9: 'OTHER', 10: 'PROD_NUM', 11: 'OTHER', 12: 'OTHER',
                        13: 'OTHER', 14: 'OTHER', 15: 'OTHER', 16: 'OTHER', 17: 'OTHER', 18: 'OTHER', 19: 'OTHER',
                        20: 'OTHER',  21: 'OTHER', 22: 'OTHER', 23: 'OTHER', 24: 'OTHER'}

manual_maps = [manual_map_A_dataset, manual_map_A_Discount_bigbuy_da, manual_map_A_Discount_bigbuy_en,
               manual_map_A_Discount_Compressed1, manual_map_A_Discount_Compressed2, manual_map_A_Discount_presta_product_2399_da,
               manual_map_A_Discount_presta_product_2403_en, manual_map_A_Discount_presta_product_2507_da,
               manual_map_A_Discount_presta_product_2507_en, manual_map_A_Discount_presta_product_2570_da,
               manual_map_A_Discount_presta_product_2570_en, manual_map_A_Discount_presta_product_2662_da,
               manual_map_A_Discount_presta_product_2662_en, manual_map_A_Discount_presta_product_2678_da,
               manual_map_A_Discount_presta_product_2678_en, manual_map_A_Discount_presta_product_3046_da,
               manual_map_A_Discount_presta_product_3046_en, manual_map_Bisgaard, manual_map_Scand, manual_map_items,
              manual_map_Joha, manual_map_Modern_classic_upstart, manual_map_PIF, manual_map_prisliste, manual_map_VAREFIL,
               manual_map_Parcellet]

In [6]:
for i in range(len(list_files)):
    evaluate(i)
#evaluate(25)


________________________________________________________________________
----------118139_revised.csv----------
----------Mapping column 1 of 30----------
Model 1 Pred: OTHER
Model 2 Pred: PRICE SIZE
Model 2b Pred: NAME
Data: SG3973, Class Prediction: OTHER
Model 1 Pred: OTHER
Model 2 Pred: PRICE SIZE
Model 2b Pred: NAME
Data: SG3982, Class Prediction: OTHER
Model 1 Pred: OTHER
Model 2 Pred: PRICE SIZE
Model 2b Pred: NAME
Data: SG3975, Class Prediction: OTHER
Model 1 Pred: OTHER
Model 2 Pred: PRICE SIZE
Model 2b Pred: NAME
Data: SG4003, Class Prediction: OTHER
Model 1 Pred: OTHER
Model 2 Pred: PRICE SIZE
Model 2b Pred: NAME
Data: SG3973, Class Prediction: OTHER
Model 1 Pred: OTHER
Model 2 Pred: PRICE SIZE
Model 2b Pred: NAME
Data: SG3935, Class Prediction: OTHER
Model 1 Pred: OTHER
Model 2 Pred: PRICE SIZE
Model 2b Pred: NAME
Data: SG4009, Class Prediction: OTHER
Model 1 Pred: OTHER
Model 2 Pred: PRICE SIZE
Model 2b Pred: NAME
Data: SG4009, Class Prediction: OTHER
Model 1 Pred: OTHER
M

KeyboardInterrupt: 

In [ ]:

print('OVERALL', percent_list)
average_total_autoperf = statistics.mean(percent_list)
print('Average automation percentage:', average_total_autoperf)
total_correct = sum(correct_amounts)
total_columns = sum(column_amounts)
column_percent = total_correct/total_columns*100
print('Correct:{correct} out of {columns}'.format(correct=total_correct, columns=total_columns))
print('Total Correct Columns Percent: {colpercent}'.format(colpercent=column_percent))


